In [ ]:
from soview import *
import pandas as pd

In [ ]:
path = 'zhiyuanyuan/PUBDT/st/spatial-datasets/data/2018_starmap_3D_cortex'
h5ad_path = 'zhiyuanyuan/PUBDT/st/starmap/h5ad'

In [ ]:
location = pd.read_csv(f'{path}/cell_locations/STARmap_3D_data_cell_locations.txt',delimiter='\t',header=None)

In [ ]:
location

In [ ]:
count = sc.read_csv(f'{path}/count_matrix/STARmap_3D_data_expression.txt',delimiter='\t').transpose()

In [ ]:
spatial_mat = []
for i in range(count.shape[0]):
    idx = int(count.obs_names[i].split('_')[1])
    cord = location.iloc[idx-1]
    spatial_mat.append([cord[0],cord[1],cord[2]])
    # break
spatial_mat = np.array(spatial_mat)

In [ ]:
count.obsm['spatial'] = spatial_mat



In [ ]:
count.write_h5ad(f'{h5ad_path}/data_3D.h5ad')

In [ ]:
sc.pp.normalize_per_cell(count)
sc.pp.neighbors(count)
sc.tl.umap(count)
sc.tl.leiden(count)

In [ ]:
count.obsm['spatial_xy'] = count.obsm['spatial'][:,[0,1]]

In [ ]:
sc.pl.embedding(count,basis='spatial_xy')

In [ ]:
sns.displot(adata.obsm['spatial'][:,1])

In [ ]:
adata = sc.read_h5ad(f'{h5ad_path}/data_3D.h5ad')

In [ ]:
adata.obs['x'] = adata.obsm['spatial'][:,0]
adata.obs['y'] = adata.obsm['spatial'][:,1]
adata.obs['z'] = adata.obsm['spatial'][:,2]

In [ ]:
adata.write_h5ad(f'{h5ad_path}/data_3D.h5ad')

In [ ]:
z_min = adata.obsm['spatial'][:,2].min()
z_max = adata.obsm['spatial'][:,2].max()
# z_step = 10

In [ ]:
z_min

In [ ]:
z_max

In [ ]:
z_step_range = np.arange(10,50,5)
# z_step_range = [5,10,15]
for z_step in z_step_range:
    z_range = np.arange(z_min,z_max,z_step)    
    for i in range(len(z_range)-1):
        a_sub = adata[adata.obsm['spatial'][:,2]>=z_range[i]]
        a_sub = a_sub[a_sub.obsm['spatial'][:,2]<z_range[i+1]]
        a_sub.write_h5ad(f'{h5ad_path}/data_2D_zstep{z_step}_{i}.h5ad')
        print(z_step,a_sub.shape)
    # if adata.obs['z']>=z_range[i] and adata.obs['z']<z_range[i+1]:
        

In [ ]:
from soview import *
import shutil
import os
import gc

root_path = 'zhiyuanyuan/projects/SODB/'
source_h5ad_path = h5ad_path
biotech_category = 'Spatial_Transcriptomics'
biotech = 'STARmap'
# 多个单词要加下划线
dataset_short = 'Wang2018three'

h5ad_list = os.listdir(source_h5ad_path)
out_dataname_list = []
out_h5ad_path_list = []
out_feature_dict_list = []
N_Unit_list = []
N_Gene_list = []
for hl in h5ad_list:
    if not hl.endswith('.h5ad'):
        continue
    # source_h5ad_data = hl.split('.')[0]
    source_h5ad_data = hl[0:-5]


    source_h5ad = f'{source_h5ad_path}/{source_h5ad_data}.h5ad'

    data_name = source_h5ad_data
    
    import os

    path_rel = f'demo_data/{biotech_category}/{biotech}/{dataset_short}/{data_name}'
    path_abs = root_path+path_rel
    os.makedirs(path_abs, exist_ok=False)
    print("The new directory is created!")
    shutil.copyfile(source_h5ad, f'{path_abs}/data.h5ad')
    
    
    data_use = path_rel
    adata = ad.read_h5ad(f'{root_path}{data_use}/data.h5ad')
    print(adata.X)
    print(f'h5ad path: {data_use}/data.h5ad')
    
    adata.obsm['spatial'] = np.array(adata.obsm['spatial'][:,[0,1]])
    
    adata.layers["counts"] = adata.X.copy()
    
    # if np.sum(adata.X)==np.sum(adata.X.astype('int')):
    if True:
        # is all the element in adata.X are integer?
    
        sc.pp.normalize_per_cell(adata)
        # sc.pp.log1p(adata)
    if adata.shape[1]>=2000:
        sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
    else:
        pass
    

    # sc.pp.pca(adata)
    sc.pp.neighbors(adata)
    sc.tl.leiden(adata)
    
    sc.pl.embedding(adata,basis='spatial',color='leiden')
    sc.tl.umap(adata,n_components=3)
    SOViewer_plot(adata,embedding_use='X_umap',dot_size=10)
    print(f'View path: {data_use}/View_UMAP_LAB.pdf')
    
    gene_list = np.array(adata.var_names).tolist()
    obs_list = np.array(adata.obs.keys()).tolist()
    obsm_list = []
    for obsm in np.array(adata.obsm.keys()).tolist():
        for i in range(adata.obsm[obsm].shape[1]):
            cur_obsm = f'{obsm}@{i}'
            obsm_list.append(cur_obsm)
            
    feature_dict = {
        'gene':gene_list,
        'obs':obs_list,
        'obsm':obsm_list
    }
    
    import pickle
    with open(f'{root_path}{data_use}/feature_dict.pickle', 'wb') as handle:
        pickle.dump(feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    
    adata.write_h5ad(f'{root_path}{data_use}/data.h5ad')
    out_dataname_list.append(source_h5ad_data)
    out_h5ad_path_list.append(f'{data_use}/data.h5ad')
    out_feature_dict_list.append(f'{data_use}/feature_dict.pickle')
    N_Unit_list.append(adata.shape[0])
    N_Gene_list.append(adata.shape[1])
    
    print(source_h5ad_data)
    print(f'h5ad path: {data_use}/data.h5ad')
    print(f'把feature_dict写为: {data_use}/feature_dict.pickle')
    print(f'N_Unit: {adata.shape[0]}')
    print(f'N_Gene: {adata.shape[1]}')
    del adata
    gc.collect()

In [ ]:
np.array(out_dataname_list).dump('out_dataname_list.npy')
np.array(out_h5ad_path_list).dump('out_h5ad_path_list.npy')
np.array(out_feature_dict_list).dump('out_feature_dict_list.npy')
np.array(N_Unit_list).dump('N_Unit_list.npy')
np.array(N_Gene_list).dump('N_Gene_list.npy')

In [ ]:
1+1

In [ ]:
for i in out_dataname_list:
    print(i)

In [ ]:
for i in out_h5ad_path_list:
    print(i)

In [ ]:
for i in out_feature_dict_list:
    print(i)

In [ ]:
for i in N_Unit_list:
    print(i)

In [ ]:
for i in N_Gene_list:
    print(i)